In [2]:
#Modelo 3
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# 1. Descargar datos históricos del activo
activo = "AAPL"  # Ticker del activo
periodo = "1y"
datos = yf.Ticker(activo).history(period=periodo)

# 2. Calcular los retornos promedio (Average) y la desviación estándar (std)
datos['Average'] = datos[['High', 'Low']].mean(axis=1).pct_change()
datos['std'] = datos['Average'].rolling(window=5).std()
datos['log_std'] = np.log(datos['std'])
datos['log_Average'] = np.log(datos['Average'].replace(0, np.nan).dropna())

# Eliminar valores NaN para los ajustes
datos.dropna(inplace=True)

# --- MODELO LOGARITMICO ---
# Ajustar el modelo exponencial transformado con logaritmos
X_exp_log = sm.add_constant(datos['Average'])  # Log(σ) = Log(α) + β * Average
modelo_exp_log = sm.OLS(datos['log_std'], X_exp_log).fit()

# Coeficientes del modelo exponencial con logaritmos
beta_exp_log = modelo_exp_log.params[1]

# Predicciones del modelo exponencial con logaritmos
datos['log_std_pred_exp'] = modelo_exp_log.params[0]+ beta_exp_log * datos['Average']


# --- GRAFICAR LOS RESULTADOS ---
fig = go.Figure()

# Datos observados
fig.add_trace(go.Scatter(x=datos['Average'], y=datos['std'],
                         mode='markers', name='Datos Observados',
                         marker=dict(size=8, color='blue', opacity=0.7)))


# Definir x_vals como un rango de valores basado en la columna 'Average'
x_vals = np.linspace(datos['Average'].min(), datos['Average'].max(), 100)

# Línea del modelo exponencial
y_vals_exp = np.exp(modelo_exp_log.params[0] + beta_exp_log * x_vals)
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_exp, mode='lines', name='Modelo Exponencial',
                         line=dict(color='green')))

# Configuración del gráfico
fig.update_layout(title="Modelos Polinomial y Exponencial | Desv. Estándar vs Retornos Promedio",
                  xaxis_title="Retornos Promedio (Average)",
                  yaxis_title="Desviación Estándar (std)",
                  template="plotly_white")

fig.show()

# --- RESULTADOS ---
print("Resultados del Modelo Polinomial:")
print(f"Intercepto (α): {alpha_poly_log:.4f}")
print(f"Coeficiente Lineal (β1): {beta_1_poly_log:.4f}")
print(f"Coeficiente Cuadrático (β2): {beta_2_poly_log:.4f}")
print(f"Resumen del Modelo Polinomial:\n{modelo_poly_log.summary()}")

print("\nResultados del Modelo Exponencial:")
print(f"Intercepto (α): {alpha_exp_log:.4f}")
print(f"Coeficiente Exponencial (β): {beta_exp_log:.4f}")


C:\Users\retir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\retir\AppData\Local\Temp\ipykernel_13456\585318426.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta_exp_log = modelo_exp_log.params[1]
C:\Users\retir\AppData\Local\Temp\ipykernel_13456\585318426.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datos['log_std_pred_exp'] = modelo_exp_log.params[0]+ beta_exp_log * datos['Average']
C

Resultados del Modelo Polinomial:


NameError: name 'alpha_poly_log' is not defined

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Coeficientes del modelo exponencial con logaritmos
beta_exp_log = modelo_exp_log.params[1]

# Generar valores de x
x_vals = np.linspace(datos['Average'].min(), datos['Average'].max(), 500)

# Calcular valores de y para el modelo exponencial con logaritmos
log_std_pred_exp = modelo_exp_log.params[0] + beta_exp_log * x_vals
std_pred_exp = np.exp(log_std_pred_exp)  # Transformación inversa para obtener valores positivos

# Filtrar valores de x y y que no superen el rango de 0.025 y sean positivos
rango_mask_exp_log = (std_pred_exp > 0) & (std_pred_exp <= 0.025)

# Crear diccionario con los datos filtrados
datos_filtrados = {
    'Average': x_vals[rango_mask_exp_log],
    'Risk': std_pred_exp[rango_mask_exp_log]
}

# Convertir a DataFrame
df_filtrado = pd.DataFrame(datos_filtrados)

# Calcular máximos y mínimos, acotación de la ecuación.
max_average = df_filtrado['Average'].max()
min_average = df_filtrado['Average'].min()
max_risk = df_filtrado['Risk'].max()
min_risk = df_filtrado['Risk'].min()

# Mostrar resultados
# Crear diccionario con los resultados
resultados_log = {
        'average': [max_average,min_average],
        'max_risk': [max_risk, min_risk]
        }

C:\Users\retir\AppData\Local\Temp\ipykernel_13456\751472180.py:6: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\retir\AppData\Local\Temp\ipykernel_13456\751472180.py:12: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [29]:
df_filtrado

,Average,Risk
0,0.000069,0.007251
1,0.000216,0.007275
2,0.000364,0.007300
3,0.000511,0.007325
4,0.000658,0.007349
...,...,...
362,0.053431,0.024595
363,0.053578,0.024678
364,0.053725,0.024761
365,0.053873,0.024845


In [33]:
resultados_modelo_3 = resultados_log

TypeError: unhashable type: 'dict'